# Ingest circuit.csv

### read data from csv file

In [0]:
dbutils.notebook.run('../Mounting_storage', 60)

In [0]:
#df_circuits = spark.read.csv('/mnt/blobstorage/circuits.csv', header=True)
#display(df_circuits.head(5))

In [0]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DoubleType

In [0]:
circuits_schema = StructType(fields=[StructField("circuitId", IntegerType(), nullable=False),
                                     StructField("circuitRef", StringType(), nullable=True),
                                     StructField("name", StringType(),  nullable=True),
                                     StructField("location", StringType(),  nullable=True),
                                     StructField("country", StringType(),  nullable=True),
                                     StructField("lat", DoubleType(),  nullable=True),
                                     StructField("lng", DoubleType(),  nullable=True),
                                     StructField("alt", IntegerType(),  nullable=True),
                                     StructField("url", StringType(),  nullable=True)])

In [0]:
df_circuits = spark.read.csv('/mnt/blobstorage/circuits.csv', header=True, schema=circuits_schema)
display(df_circuits.head(5))

In [0]:
df_circuits.printSchema()

In [0]:
#from pyspark.sql.functions import col
#
## Filter out rows where circuitId is null
#df_circuits = df_circuits.filter(col("circuitId").isNotNull())

In [0]:
#circuits_select = df_circuits.select("circuitId", "circuitRef", "name", "location", "country", "lat", "lng", "alt")
#
#display(circuits_select)

### Renaming the required columns

In [0]:
from pyspark.sql.functions import col

circuits_select = df_circuits.select(col("circuitId"), col("circuitRef"), col("name"), col("location"), col("country").alias("race_country"), col("lat"), col("lng"), col("alt"))

display(circuits_select)

In [0]:
circuits_select_df = circuits_select.withColumnRenamed("circuitId","circuit_id")\
    .withColumnRenamed("circuitRef","circuit_Ref")\
        .withColumnRenamed("name","circuit_name")\
        .withColumnRenamed("lat","latitude")\
            .withColumnRenamed("lng","longitude")\
                .withColumnRenamed("alt","altitude")

In [0]:
display(circuits_select_df)

### Adding a column with ingestion Date

In [0]:
#from pyspark.sql.functions import current_timestamp,lit
#
#circuits_select_df = circuits_select_df.withColumn("ingestion_date",current_timestamp())\
#    .withColumn("env",lit("Production"))
#display(circuits_select_df)

In [0]:
from pyspark.sql.functions import current_timestamp

circuits_select_df = circuits_select_df.withColumn("ingestion_date",current_timestamp())
display(circuits_select_df)

In [0]:
circuits_select_df.write.mode("overwrite").parquet('/mnt/blobstorage/circuits')

In [0]:
%fs
ls mnt/blobstorage/circuits